# AI Learns to Play Connect 4
#### Jordan Yeomans - 2018

## Part 3 - Make A Winning Move

### What's Next?

I noticed that the bot's playing randomly made some pretty dumb moves, which makes sense. But to speed up the training process I wanted to include 1 modification. If the bot can find a winning move, lets make that move.

Maybe a bit later down the track we can learn to play from absolute scratch!

These changes are to the bot file riddles.io is using to playy. In particular we are changing the function: Skip to the end to copy and paste the entire thing! You'll also need to add the connect 4 function (Step 6).

- make_move(state)

#### Step 1: Record Data

Just like in part 1, we need to record the historical data during each move. We check the state.round, and if it is 0 we need top make a new historical file. If not, we can load the current file.

I know this isn't the most efficient way (Saving/Loading everytime) but since they are so small it works fine

PS. For this document I have split the function into different parts so that the notebook is easier to read and complies. In the actual code this is 1 function

In [1]:
def make_move(state):
    
    # Convert field state into a numpy array
    field_state = np.array(state.field.field_state)

    # Recording Data
    if state.round == 0:
        game_history = np.zeros((30, 6, 7))
    else:
        game_history = np.load(state.name)
    
    ## Record Game History. 1 = Yellow Tokens, -1 = Red Tokens, 0 = Empty Spot
    # Iterate over all rows
    for row in range(6):
        yellow_idx = np.where(field_state[row] == '1')[0]
        red_idx = np.where(field_state[row] == '0')[0]

        game_history[state.round][row][yellow_idx] = 1
        game_history[state.round][row][red_idx] = -1

#### Step 2: Chose A Random Valid Move

We aren't sure if we can win yet, but if we can't we still need to make a move. At this stage lets choose a random column that is valid (top position is empty)

Then, if we can win, we can change our mind and place the token in that column.

1. Find the places in the top row that are empty
2. Shuffle the index's randomly
3. Select the first index as the randomly chosen, valid, column

In [2]:
def make_move_p1():
    
    # Choose a random move out of all valid columns
    valid_idx = np.where(game_history[state.round][0] == 0)[0]
    np.random.shuffle(valid_idx)
    move = valid_idx[0]

#### Step 3: Work Out Who Moved First

Before we move, we should see if we can win this. Before we can do that, we need to know who's turn it is. Since the game doesn't say who's turn it is we need to figure it out ourselves.

If the state.round = 1 we can check how many tokens are on the board, and what number they add up to. From that, we can figure out who's turn it is.

Remember, this bot needs to work with playing as both red and yellow.

In [3]:
def make_move_p2():
    
    ## Before we move, let's check if there is a move we can make to win!
    
    # 1. Determine who moved first
    if state.round == 1:
        token_idx = np.where(field_state[5] != '.')[0]
        r1_tokens = np.array(field_state[5][token_idx]).astype(float)

        if np.sum(r1_tokens) == 1 and r1_tokens.shape[0] == 2:    # Red First, Red See's
            state.first = 'Red'
        elif np.sum(r1_tokens) == 1 and r1_tokens.shape[0] == 3:  # Red First, Yellow See's
            state.first = 'Red'

        elif np.sum(r1_tokens) == 1 and r1_tokens.shape[0] == 1:  # Yellow First, Red See's
            state.first = 'Yellow'
        elif np.sum(r1_tokens) == 2 and r1_tokens.shape[0] == 3:  # Yellow First, Yellow See's
            state.first = 'Yellow'

#### Step 4: Determine Who's Turn It Is

If we add up the sum of the board history for the current round and count the tokens we can figure out who's turn it is.

1. If the Sum is -1: There is more Red Tokens on the board, therefore it it yellow's turn
2. If the Sum is 0, and Yellow moved first: There is an even number of tokens on the board, and yellow was first, so it's yellow's turn
3. If Sum is 0, and red moved first: Same as 2. except red was first, so it's red's turn

We want to make a new board that represents what the current player is seeing regardless of being red or yellow. We will call this the me_board and a 1 is the view of the bot and -1 is the enemy. We will make the me_board next

In [4]:
def make_move_p3():
    
    # 2. Determine who's turn it is
    # If sum = -1, More Red Tokens -> Yellows's turn
    if np.sum(game_history[state.round]) == -1:
        me_idx = np.where(game_history[state.round] == 1)     
        enemy_idx = np.where(game_history[state.round] == -1)

    # If sum = 0 -> Even # Tokens & Yellow moved first --> Mean's it's Yellow's turn
    elif np.sum(game_history[state.round]) == 0 and state.first == 'Yellow':
        me_idx = np.where(game_history[state.round] == 1)

        enemy_idx = np.where(game_history[state.round] == -1)
    # If sum = 0, Even # Tokens & Red moved first
    elif np.sum(game_history[state.round]) == 0:
        me_idx = np.where(game_history[state.round] == -1)
        enemy_idx = np.where(game_history[state.round] == 1)

#### Step 5: Create The Me Board

Since this code is meant to play as both red and yellow players, we need to know what the current player is seeing. We have stored the index's of the current players tokens stored as a 1. The enemy tokens are stored as a -1

1. Make a new board of zero's
2. Assign all me-indexs as 1
3. Assign all enemy indexes as -1


In [5]:
def make_move_p4():
    
    # 2. Organise the board so that 1 = Me, -1 = Enemy.
    me_board = np.zeros_like(game_history[state.round])
    me_board[me_idx] = 1
    me_board[enemy_idx] = -1

#### Step 6: Create A Connect 4 Function

Before we can find a winning column, we need to make a function to determine a winning move!

Let's make a function that determines if the board provided contains connect 4.

1. Set a win-flag to False, this will be turned to True if we find a winning move which can be returned

2. Check the diagonal (Bottom Left -> Top Right). To do so, we iterate over all options for p1 and the corresponding points p2, p3, p4. We can then sum p1, p2, p3, p4 together and if the sum is 4, we have connect 4!

3. Do the same for the opposite diagonal
4. Check if any row's win
5. Check if any columns win

It might be easier to see a picture of the index's we are checking. For anyone new to python, index's start at 0 so the "first" column would be index 0.

In [11]:
from IPython.display import Image
from IPython.core.display import HTML 

Image(url= "https://i.imgur.com/w3UoVtp.png", width=800)

##### For anyone unfamiliar with Numpy array's
Notice the notation to access the columns. It can be a bit tricky. It took me a while to get to the point I didn't need to google everytime!

section = board[:, col][p1:p4+1]

With this command we are saying: 
1. board -> From The Board
2. [ : , col] -> [Give me all rows : ,  For Column #]
3. [p1:p4+1] -> [Now From that Array, give me Index P1 up to P4]



In [6]:
def four_in_a_row(board):

    win_flag = False

    # Check Diagonal (P1 = Bottom Left - P4 = Top Right)
    for p1_col in range(4):
        p2_col = p1_col + 1
        p3_col = p2_col + 1
        p4_col = p3_col + 1

        for p1_row in range(3, 6):
            p2_row = p1_row - 1
            p3_row = p2_row - 1
            p4_row = p3_row - 1

            p1 = board[p1_row][p1_col]
            p2 = board[p2_row][p2_col]
            p3 = board[p3_row][p3_col]
            p4 = board[p4_row][p4_col]

            if np.sum([p1, p2, p3, p4]) == 4:
                win_flag = True

    # Check Diagonal (P1 = Top Left - P4 = Bottom Right)
    for p1_col in range(3):
        p2_col = p1_col + 1
        p3_col = p2_col + 1
        p4_col = p3_col + 1

        for p1_row in range(3):
            p2_row = p1_row + 1  # Careful, we swap sign to +
            p3_row = p2_row + 1  # Careful, we swap sign to +
            p4_row = p3_row + 1  # Careful, we swap sign to +

            p1 = board[p1_row][p1_col]
            p2 = board[p2_row][p2_col]
            p3 = board[p3_row][p3_col]
            p4 = board[p4_row][p4_col]

            if np.sum([p1, p2, p3, p4]) == 4:
                win_flag = True

    # Check for row win
    for row in range(board.shape[0]):
        for p1 in range(4):
            p4 = p1 + 3
            section = board[row][p1:p4+1]
            if np.sum(section) == 4:
                win_flag = True

    # Check for column win
    for col in range(board.shape[1]):
        for p1 in range(3):
            p4 = p1 + 3
            section = board[:, col][p1:p4+1]
            if np.sum(section) == 4:
                win_flag = True

    return win_flag

#### Step 7: Check If Any Moves Would Win

Now we have a function to determine if a particular board Contains connect 4 (From the view of the current player only) let's see if we can win this game

1. Iterate over all valid columns
2. Redcord all the empty rows in that column
3. Get the maximum row index, that is going to be the lowest position. Place a token in this position
4. Send the board to the connect 4 function
5. If the win_flag is True, that placing a token in that column will win the game. Break the function and let's make that move


If there is no winning moves, we will keep the move we calculated at the very beginning. Before we do, we need to save the game_history

In [7]:
def make_move_p5():
    
    # 3. Check if any columns are winning columns:
    for col_idx in valid_idx:
        row_idx = np.where(me_board[:, col_idx] == 0)[0]
        row_idx = np.max(row_idx)
        me_board[row_idx][col_idx] = 1
        win_flag = four_in_a_row(me_board)
        if win_flag is True:
            move = col_idx
            break

    np.save(state.name, game_history)

    return 'place_disc {}'.format(move)

#### Part 4 Is Next - Organising The Data For A Neural Network

Before we go, here is the complete code for the make_move() function. You can copy this to your bot file

In [8]:
def make_move(state):

    # Convert field state into a numpy array
    field_state = np.array(state.field.field_state)

    # Recording Data
    if state.round == 0:
        game_history = np.zeros((30, 6, 7))
    else:
        game_history = np.load(state.name)

    ## Record Game History. 1 = Yellow Tokens, -1 = Red Tokens, 0 = Empty Spot
    # Iterate over all rows
    for row in range(6):
        yellow_idx = np.where(field_state[row] == '1')[0]
        red_idx = np.where(field_state[row] == '0')[0]

        game_history[state.round][row][yellow_idx] = 1
        game_history[state.round][row][red_idx] = -1

    # Choose a random move out of all valid columns
    valid_idx = np.where(game_history[state.round][0] == 0)[0]
    np.random.shuffle(valid_idx)
    move = valid_idx[0]

    # Determine who moved first
    if state.round == 1:
        token_idx = np.where(field_state[5] != '.')[0]
        r1_tokens = np.array(field_state[5][token_idx]).astype(float)

        if np.sum(r1_tokens) == 1 and r1_tokens.shape[0] == 2:    # Red First, Red See's
            state.first = 'Red'
        elif np.sum(r1_tokens) == 1 and r1_tokens.shape[0] == 3:  # Red First, Yellow See's
            state.first = 'Red'

        elif np.sum(r1_tokens) == 1 and r1_tokens.shape[0] == 1:  # Yellow First, Red See's
            state.first = 'Yellow'
        elif np.sum(r1_tokens) == 2 and r1_tokens.shape[0] == 3:  # Yellow First, Yellow See's
            state.first = 'Yellow'

        if state.first is None:
            print(np.sum(r1_tokens))

    ## Before we move, let's check if there is a move we can make to win!
    
    # 1. Determine who's turn it is
    # If sum = -1, More Yellow Tokens -> Red's turn
    if np.sum(game_history[state.round]) == -1:
        me_idx = np.where(game_history[state.round] == 1)     # These might be wrong
        enemy_idx = np.where(game_history[state.round] == -1) # These might be wrong

    # If sum = 0 -> Even # Tokens & Yellow moved first --> Mean's it's Yellow's turn
    elif np.sum(game_history[state.round]) == 0 and state.first == 'Yellow':
        me_idx = np.where(game_history[state.round] == 1)

        enemy_idx = np.where(game_history[state.round] == -1)
    # If sum = 0, Even # Tokens & Red moved first
    elif np.sum(game_history[state.round]) == 0:
        me_idx = np.where(game_history[state.round] == -1)
        enemy_idx = np.where(game_history[state.round] == 1)

    # 2. Organise the board so that 1 = Me, -1 = Enemy.
    me_board = np.zeros_like(game_history[state.round])
    me_board[me_idx] = 1
    me_board[enemy_idx] = -1

    # 3. Check if any columns are winning columns:
    for col_idx in valid_idx:
        row_idx = np.where(me_board[:, col_idx] == 0)[0]
        row_idx = np.max(row_idx)
        me_board[row_idx][col_idx] = 1
        win_flag = four_in_a_row(me_board)
        if win_flag is True:
            move = col_idx
            break

    np.save(state.name, game_history)

    return 'place_disc {}'.format(move)